In [1]:
from music21 import *
import numpy, collections
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pickle
import glob

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html
Using TensorFlow backend.


In [10]:
bigdurations=['2.0','3.0','1.5','1.75','12.0','5/3','4.0','2/3','8.0','7.5','6.0','4/3',
              '3.5','5.0','3.25','5.0','10/3','2.25','3.75','1.25']

midledurations = ['0.5','1.0','0.75']

shortdurations=['0.25','1/3','0.0']

dura=[]

notes = []
duration = []

for midfile in glob.glob('midi_source/beethoven/sonata14/*.mid'):

    mid = converter.parse(midfile)

    notes_to_parse = None

    parts = instrument.partitionByInstrument(mid)

    if parts:  # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()

    else:  # file has notes in a flat structure
        notes_to_parse = mid.flat.notes
        print('teste')

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            a = str(element.pitch)
            a += ("|")
            if(str(element.duration.quarterLength) in bigdurations):
                a +='2.0'
            elif(str(element.duration.quarterLength) in midledurations):
                a +='1.0'
            elif(str(element.duration.quarterLength) in shortdurations):
                a +='0.25'
            else:
                a +=str(element.duration.quarterLength)
            
            
            
            notes.append(a)
            

        elif isinstance(element, chord.Chord):
            chord_pitches=[]


            for p in element.pitches:
                chord_pitches.append(p.midi)


            chord_pitches = str(chord_pitches).strip('[]')
            chord_pitches += ("|")

            if(str(element.duration.quarterLength) in bigdurations):
                chord_duration = '2.0'
            elif(str(element.duration.quarterLength) in midledurations):
                chord_duration ='1.0'
            elif(str(element.duration.quarterLength) in shortdurations):
                chord_duration ='0.25'
            else:
                chord_duration = str(element.duration.quarterLength)
                
            chord_duration = str(element.duration.quarterLength)
            chord_pitches += chord_duration
            
            chord_pitches += ("|chord")


            notes.append(chord_pitches)
            
            with open('data/sonata14/controlduration/sonata14-full', 'wb') as path:
                pickle.dump(notes, path)


    
#print(notes)
print(collections.Counter(notes))

Counter({'E-3|0.25': 268, 'G#3|0.25': 265, 'G#2|0.25': 263, 'C#4|0.25': 198, 'C#5|0.25': 164, 'G#4|0.25': 156, 'C#3|0.25': 147, 'E4|0.25': 142, 'F#4|0.25': 135, 'G#5|0.25': 134, 'A4|0.25': 104, 'E3|0.25': 95, 'B3|0.25': 93, 'E-4|0.25': 85, 'G#1|0.25': 83, 'A3|0.25': 75, 'F#3|0.25': 69, 'B2|0.25': 65, 'E5|0.25': 64, 'B4|0.25': 63, 'C#2|0.25': 57, 'B-3|0.25': 56, 'E-5|0.25': 55, 'B5|0.25': 48, 'F#5|0.25': 48, 'C5|0.25': 47, 'A5|0.25': 44, 'D5|0.25': 44, 'G5|0.25': 42, 'C4|0.25': 39, 'G4|0.25': 35, 'G#4|1.0': 32, 'G2|0.25': 31, 'B-5|0.25': 30, 'E-4|1.0': 28, 'A2|0.25': 27, 'F#4|1.0': 27, 'E4|1.0': 26, 'C3|0.25': 26, 'C#4|1.0': 25, 'D4|0.25': 25, 'A1|0.25': 25, 'F4|0.25': 25, '68, 56|0.25|chord': 24, 'G3|0.25': 22, 'D3|0.25': 22, 'G#2|2.0': 22, 'B-4|0.25': 22, '56, 44|0.25|chord': 21, 'C4|1.0': 21, 'F3|0.25': 19, 'C#5|1.0': 19, 'C#6|0.25': 19, 'E2|0.25': 19, 'B-2|0.25': 19, 'F#2|0.25': 18, '63, 51|0.25|chord': 18, '66, 54|0.25|chord': 17, '61, 49|0.25|chord': 16, 'E-2|0.25': 16, 'B4|1.0': 

In [9]:
 '''with open('data/beethoven', 'rb') as path:
        notes = pickle.load(path)
     '''   
chars = sorted(list(set(notes)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(notes)
n_vocab = len(chars)
print ("Total Characters: ")
print(n_chars)
print ("Total Vocab: ")
print(n_vocab)


Total Characters: 
5715
Total Vocab: 
927


In [ ]:
seq_length = 1000
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = notes[i:i + seq_length]
    seq_out = notes[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print( "Total Patterns: ") 
print(n_patterns)

In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model

In [ ]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
# define the checkpoint
filepath="w{epoch:02d}-{loss:.4f}-moonlight.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=200, batch_size=100, callbacks=callbacks_list)